In [1]:
import os
import warnings
import pandas as pd
from prophet import Prophet

warnings.filterwarnings("ignore")


D:\amdox technology\Crypto time series analysis\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("📍 Current working directory:", os.getcwd())

DATA_PATH = "crypto_master_cleaned.csv"

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"❌ File not found: {os.path.abspath(DATA_PATH)}")

df = pd.read_csv(DATA_PATH)

# Normalize columns
df.columns = df.columns.str.strip().str.lower()

# Convert date
df["date"] = pd.to_datetime(df["date"])

df.head()


📍 Current working directory: D:\amdox technology\Crypto time series analysis\notebooks


,date,open_price,high_price,low_price,close_price,volume,crypto
0,2020-12-31,0.184034,0.185749,0.176594,0.181397,1132268397,ADA
1,2021-01-01,0.181382,0.184246,0.172022,0.175350,1122218004,ADA
2,2021-01-02,0.175359,0.184253,0.169233,0.177423,1408849504,ADA
3,2021-01-03,0.177382,0.208679,0.173376,0.204995,2303857909,ADA
4,2021-01-04,0.205236,0.239661,0.194450,0.224762,3260699086,ADA


In [3]:
FORECAST_STEPS = 30
prophet_rows = []

coins = df["crypto"].unique()

for coin in coins:
    print(f"🔮 Prophet forecasting {coin}")

    coin_df = df[df["crypto"] == coin][["date", "close_price"]].copy()

    # Prophet requires specific column names
    coin_df.rename(
        columns={"date": "ds", "close_price": "y"},
        inplace=True
    )

    try:
        model = Prophet(
            daily_seasonality=True,
            yearly_seasonality=True,
            weekly_seasonality=False
        )

        model.fit(coin_df)

        future = model.make_future_dataframe(periods=FORECAST_STEPS)
        forecast = model.predict(future)

        forecast_tail = forecast.tail(FORECAST_STEPS)

        for _, row in forecast_tail.iterrows():
            prophet_rows.append({
                "date": row["ds"],
                "crypto": coin,
                "forecast_price": float(row["yhat"]),
                "model": "PROPHET"
            })

    except Exception as e:
        print(f"❌ Prophet failed for {coin}: {e}")


🔮 Prophet forecasting ADA


18:45:47 - cmdstanpy - INFO - Chain [1] start processing
18:45:47 - cmdstanpy - INFO - Chain [1] done processing


🔮 Prophet forecasting BCH


18:45:48 - cmdstanpy - INFO - Chain [1] start processing
18:45:49 - cmdstanpy - INFO - Chain [1] done processing


🔮 Prophet forecasting BNB


...

18:45:59 - cmdstanpy - INFO - Chain [1] start processing
18:45:59 - cmdstanpy - INFO - Chain [1] done processing


🔮 Prophet forecasting SOL


18:46:00 - cmdstanpy - INFO - Chain [1] start processing
18:46:00 - cmdstanpy - INFO - Chain [1] done processing
18:46:00 - cmdstanpy - ERROR - Chain [1] error: code '1' Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
18:46:00 - cmdstanpy - INFO - Chain [1] start processing
18:46:03 - cmdstanpy - INFO - Chain [1] done processing


🔮 Prophet forecasting XRP


18:46:04 - cmdstanpy - INFO - Chain [1] start processing
18:46:04 - cmdstanpy - INFO - Chain [1] done processing


In [4]:
prophet_df = pd.DataFrame(prophet_rows)

prophet_df.to_csv("prophet_all_coins.csv", index=False)

print("✅ Prophet forecast saved successfully")


✅ Prophet forecast saved successfully


In [5]:
print("Prophet rows:", prophet_df.shape)
prophet_df.head()


Prophet rows: (300, 4)


,date,crypto,forecast_price,model
0,2022-03-07,ADA,0.872146,PROPHET
1,2022-03-08,ADA,0.849612,PROPHET
2,2022-03-09,ADA,0.826901,PROPHET
3,2022-03-10,ADA,0.804222,PROPHET
4,2022-03-11,ADA,0.781776,PROPHET


In [6]:
prophet_df.tail()

,date,crypto,forecast_price,model
295,2022-04-01,XRP,1.005733,PROPHET
296,2022-04-02,XRP,1.044562,PROPHET
297,2022-04-03,XRP,1.084222,PROPHET
298,2022-04-04,XRP,1.124416,PROPHET
299,2022-04-05,XRP,1.164845,PROPHET
